In [1]:
#dansk bibel                            "bla bla (originalt danski)" 
#engelsk bibel "bla bla" steering homie "bla bla (men dansk)"
from huggingface_hub import hf_hub_download
from utils.steering import generate_with_steering
import fasttext
import torch
import os
from transformers import AutoModelForCausalLM
from classes.datahandling import ParallelNSPDataset
from utils.probe_confidence_intervals import model_setup
from collections import defaultdict
import pandas as pd
from fasttext.FastText import _FastText


model, tokenizer, device = model_setup("AI-Sweden-Models/gpt-sw3-356m")

model_name = "nb-nordic-lid.ftz"
language_prediction_model = fasttext.load_model(hf_hub_download("NbAiLab/nb-nordic-lid", model_name))

label, score = language_prediction_model.predict("Harry Potter är en serie fantasyromaner av författaren J.K. Rowling, som började ges", threshold=0.25)
label[0].split("__")[-1], score
bible_data = ParallelNSPDataset.from_tmx("data/bible-da-en.tmx","da","en")
bible_data[0]

found device: cpu


{'da': ('I Begyndelsen skabte Gud Himmelen og Jorden.',
  'Og Jorden var øde og tom, og der var Mørke over Verdensdybet. Men Guds Ånd svævede over Vandene.'),
 'en': ('In the beginning God created the heavens and the earth.',
  "Now the earth was formless and empty. Darkness was on the surface of the deep. God's Spirit was hovering over the surface of the waters.")}

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Path to your model files
model_path = 'gpt_gptsw3_en_no_is_da_356m_gbs1024'

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

print("Model and tokenizer loaded successfully!")


Model and tokenizer loaded successfully!


In [32]:
tokenized = tokenizer("hej med dig",return_tensors= "pt")
out = model.generate(tokenized.input_ids, pad_token_id=tokenizer.eos_token_id,max_new_tokens = 20)
print(tokenizer.decode(out[0]))

hej med dig.
-Jeg er ikke sikker på, at jeg kan klare det.
-Det er jeg
